In [2]:
import pandas as pd
import numpy as np
import os
from bokeh.io import output_notebook, show
output_notebook()
import bokeh as bh
from sklearn.preprocessing import LabelEncoder
from glob import glob
import re
from math import ceil
import xgboost as xgb
from sklearn.model_selection import GroupKFold,KFold,StratifiedKFold,train_test_split
import random
import operator
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
import fastai.tabular as ft
pd.options.display.max_columns = 1000

Loading BokehJS ...

In [3]:
train_id = pd.read_csv('train_identity.csv')
train_trans = pd.read_csv('train_transaction.csv')
train_df = train_trans.merge(train_id,how='left',on=['TransactionID'])

In [4]:
test_id = pd.read_csv('test_identity.csv')
test_trans = pd.read_csv('test_transaction.csv')
test_df = test_trans.merge(test_id,how='left',on=['TransactionID'])

In [5]:
cols = list(train_df.columns)
for i in ['isFraud']:
    cols.remove(i)

In [6]:
for f in cols:
    if train_df[f].dtype=='object':
        print(f+"-"+str(train_df[f].dtype))
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[f].values) + list(test_df[f].values))
        train_df[f] = lbl.transform(list(train_df[f].values))
        test_df[f] = lbl.transform(list(test_df[f].values))

ProductCD-object
card4-object
card6-object
P_emaildomain-object
R_emaildomain-object
M1-object
M2-object
M3-object
M4-object
M5-object
M6-object
M7-object
M8-object
M9-object
id_12-object
id_15-object
id_16-object
id_23-object
id_27-object
id_28-object
id_29-object
id_30-object
id_31-object
id_33-object
id_34-object
id_35-object
id_36-object
id_37-object
id_38-object
DeviceType-object
DeviceInfo-object


In [7]:
build_df = train_df[train_df['TransactionDT'] < 13800000].reset_index()
val_df = train_df[train_df['TransactionDT'] >= 13800000].reset_index()

In [8]:
class MetricsMeter():    
    def __init__(self, y_true, y_pred, threshold):
        self.y_true = y_true        
        self.y_pred = y_pred        
        self.thresh = threshold            
    
    def fit(self):        
        metrics = {}        
        metrics['auc'] = roc_auc_score(self.y_true, self.y_pred)                
        self.y_pred = [1 if x > self.thresh else 0 for x in self.y_pred]        
        metrics['f1'] = f1_score(self.y_true, self.y_pred,average = 'macro')        
        tn, fp, fn, tp = confusion_matrix(self.y_true, self.y_pred).ravel()                
        metrics['sensitivity/recall'] = tp / (tp+fn)
        metrics['precision'] = tp / (tp+fp)
        metrics["accuracy"] = (tp + tn) / (tp + fp + fn + tn)
        metrics['specificity'] = tn / (tn+fp)
        return metrics

In [15]:
def RunXGB(x_build,x_val,y_build,y_val,dtest,seed):
    dbuild = xgb.DMatrix(x_build,y_build)
    dval = xgb.DMatrix(x_val,y_val)
    watchlist = [(dbuild, 'train'), (dval, 'val')]
    params = {'objective': 'binary:logistic',
              'booster': 'gbtree',
              'eval_metric': 'auc',
              'nthread': 32,
              'max_depth': 6,
              'subsample': 0.8,
              'min_child_weight': 1,
              "colsample_bytree": 0.9,
              'eta': 0.08,
              'verbose_eval': True,
              'silent':1,
              'seed': seed
              }
    clf_xgb = xgb.train(params, dbuild, num_boost_round=2000, verbose_eval=50, early_stopping_rounds=30, evals=watchlist)
    pred_val = clf_xgb.predict(dval, ntree_limit=clf_xgb.best_iteration)
    pred_t = clf_xgb.predict(dtest, ntree_limit=clf_xgb.best_iteration)
    return(clf_xgb,pred_val,pred_t)

In [16]:
model1,pred_v1,pred_t1 = RunXGB(build_df[cols].iloc[:],val_df[cols].iloc[:],
build_df['isFraud'].iloc[:],val_df['isFraud'].iloc[:],xgb.DMatrix(test_df[cols].iloc[:]),seed = 9999)

[0]	train-auc:0.792718	val-auc:0.79463
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 30 rounds.
[50]	train-auc:0.899261	val-auc:0.88036
[100]	train-auc:0.926581	val-auc:0.899049
[150]	train-auc:0.936789	val-auc:0.906377
[200]	train-auc:0.945792	val-auc:0.910673
[250]	train-auc:0.95225	val-auc:0.914573
[300]	train-auc:0.957616	val-auc:0.916427
[350]	train-auc:0.961808	val-auc:0.917263
[400]	train-auc:0.965141	val-auc:0.917749
[450]	train-auc:0.968289	val-auc:0.919285
[500]	train-auc:0.971103	val-auc:0.919392
Stopping. Best iteration:
[482]	train-auc:0.970113	val-auc:0.920042



In [17]:
model2,pred_v2,pred_t2 = RunXGB(build_df[cols].iloc[:],val_df[cols].iloc[:],
build_df['isFraud'].iloc[:],val_df['isFraud'].iloc[:],xgb.DMatrix(test_df[cols].iloc[:]),seed = 99)

MemoryError: 

In [18]:
pred_df = pd.DataFrame(pred_t1 ,columns = ['isFraud'])

In [19]:
fnl = pd.concat([test_df['TransactionID'].reset_index(drop=True), pred_df], axis=1)

In [20]:
fnl.head(10)

,TransactionID,isFraud
0,3663549,0.001185
1,3663550,0.001624
2,3663551,0.003684
3,3663552,0.000704
4,3663553,0.003196
5,3663554,0.004221
6,3663555,0.020538
7,3663556,0.010617
8,3663557,0.000327
9,3663558,0.004030


In [22]:
fnl.to_csv("fraud_preds_4.csv",index=False)

In [ ]:
from google.colab import files

fnl.to_csv('fnl_part4.csv',index = False)
files.download('fnl_part4.csv')